MediaPipe is an open-source framework developed by Google that provides a comprehensive solution for building real-time perception pipelines for a wide range of applications, including object detection, facial recognition, hand tracking, pose estimation, and more. It offers pre-trained machine learning models, as well as tools and components for processing, analyzing, and interpreting various forms of media input, such as images, video streams, and sensor data.

In [1]:
# Required imports
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
import math
import tensorflow as tf
from tensorflow.keras.models import load_model
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


In [2]:
# Storing the coordinates of the points traversed by the finger 
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]


# These indexes will be used to mark the points in particular arrays of specific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

In [3]:
# Colors in BGR format Blue, Green, Red, Yellow 
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]

# Which color is selected
colorIndex = 0

In [4]:
# Paint window of 471 length, 636 breadth, 3 width , white color
paintWindow = np.zeros((471,636,3)) + 255

cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

In [5]:
#Initialise Mediapipe

# This module provides functionality for detecting and tracking hands in images or video streams.
mpHands = mp.solutions.hands

# max_num_hands=1 specifies that the model should detect at most one hand in the input
# min_detection_confidence=0.7 means hand detections with a confidence score >= 0.7 will be considered valid.
hands = mpHands.Hands(max_num_hands = 1, min_detection_confidence = 0.7)

# provides utility functions for drawing landmarks and connections on images or video frames
mpDraw = mp.solutions.drawing_utils


# model = load_model('/Air_Canvas/mp_hand_gesture')


labels = ['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']
flag = 0

In [8]:
# Initialize the webcam
cap = cv2.VideoCapture(0)
ret = True

while ret:
    # Used to capture a frame from a video
    # read() returns tow values boolean ret = True when the frame is captured successfully
    # frame contains the actual image data of the captured frame.
    ret, frame = cap.read()
    # x = heaight, y = width, c = no.of channels(1 if img is in gray scale, 3 if in RGB)
    x, y, c = frame.shape
    
    
    # Flip the along vertical axis (1 - along vertical axis, 0 means along horizontal axis)
    frame = cv2.flip(frame, 1)
    # converting img from bgr format to rgb format
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Used to draw a rectangle on an img Frame, top left coordinates, bottom right coordinates, color of rectangle, border width in pixels   
    frame = cv2.rectangle(frame, (20, 15), (96, 48), (0, 0, 0), 2)
    frame = cv2.rectangle(frame, (118, 15), (194, 48), (255, 0, 0), 2)
    frame = cv2.rectangle(frame, (216, 15), (292, 48), (0, 255, 0), 2)
    frame = cv2.rectangle(frame, (314, 15), (390, 48), (0, 0, 255), 2)
    frame = cv2.rectangle(frame, (412, 15), (488, 48), (0, 255, 255), 2)
    frame = cv2.rectangle(frame, (510, 15), (586, 48), (226, 43, 138), 2)
    frame = cv2.rectangle(frame, (612, 15), (636, 48), (255, 255, 0), 2)

    cv2.putText(frame, "CLEAR", (30, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (133, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (225, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "RED", (335, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "LM", (535, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

     if flag == 1:
        if result.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame, result.multi_hand_landmarks[0], mpHands.HAND_CONNECTIONS)
    # Get hand landmark prediction
    # result contains information such as the detected hand landmarks (e.g. keypoints representing fingertips, palm center)
    result = hands.process(framergb)
    
    className = ''
    
    # if the result conatains hand
    hand_present = result.multi_hand_landmarks
    if result.multi_hand_landmarks:
        landmarks = []
        
        for lm in result.multi_hand_landmarks[0].landmark:

                # scaling the coordinates of landmarks acc to the paintWindow because the values are in [0,1]
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)

                landmarks.append([lmx, lmy])
                
        
         # Predict gesture in Hand Gesture Recognition project
         # prediction = model.predict([landmarks]) 

         # classID = np.argmax(prediction)
         # className = classNames[classID]

        cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2, cv2.LINE_AA)
        
        fore_finger = (landmarks[8][0],landmarks[8][1])
        thumb_finger = (landmarks[4][0],landmarks[4][1])
        
        distance = math.sqrt((fore_finger[0] - thumb_finger[0])**2 + (fore_finger[1] - thumb_finger[1])**2)         
        # do not write on the paintWindow when fore_finger and thumb are closer
        if distance < 30:
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1
           
        # when the finger is in the button region
        elif fore_finger[1] <= 65:
            
            # Clear button
            if 40 <= fore_finger[0] <= 145:
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                # clearing the paint window
                paintWindow[66:,:,:] = 255
                
            elif 160 <= fore_finger[0] <= 255 : # Blue button
                    colorIndex = 0 
            elif 275 <= fore_finger[0] <= 370 : # Green button
                    colorIndex = 1
            elif 390 <= fore_finger[0] <= 485 : # Red button
                    colorIndex = 2 
            elif 505 <= fore_finger[0] <= 600 : # Yellow button
                    colorIndex = 3  
            elif 615 <= fore_finger[0] <= 635 : # Yellow button
                    if flag == 0:
                        flag = 1
                    elif flag == 1:
                        flag = 0
                        
                        
                        
                        
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(fore_finger)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(fore_finger)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(fore_finger)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(fore_finger)
    else :

        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1
            
    points = [bpoints, gpoints, rpoints, ypoints]

    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame) 
    cv2.imshow("Paint", paintWindow)

    if cv2.waitKey(1) == ord('q'):
        break

# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()